# Factorizing Semiprimes using Shor's algorithm

### **Install Quantum Rings**

In [1]:
!pip install QuantumRingsLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.3 MB/s eta 0:00:00


## Importing Libraries and Setting Up Quantum Rings Provider

This cell imports the necessary libraries for implementing Shor's algorithm using Quantum Rings. It includes:
- **QuantumRingsLib**: A quantum computing library for working with quantum rings.
- **Matplotlib and NumPy**: For visualization and numerical operations.
- **Math and Random**: For mathematical operations and random number generation.

The cell initializes the **QuantumRingsProvider** with authentication credentials and retrieves the backend named `"scarlet_quantum_rings"`. It also verifies the active account.



In [2]:
import time
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from matplotlib import pyplot as plt
import numpy as np
import math
import random
from math import gcd
import threading
import time, random

provider = QuantumRingsProvider(
    token='rings-200.YgwkLRmk6lGxpnBseHdbT9qm265I7L00',
    name='aarushh007@gmail.com'
)
backend = provider.get_backend("scarlet_quantum_rings")
provider.active_account()

{'name': 'aarushh007@gmail.com',
 'token': 'rings-200.YgwkLRmk6lGxpnBseHdbT9qm265I7L00',
 'max_qubits': '200'}

## Defining the Number of Shots

This cell sets the number of shots (repetitions of the quantum circuit execution) to **2048**.  


In [3]:
shots = 2048

## Modular Exponentiation Circuit (Simplified)

This function **`modular_exponentiation`** implements a simplified version of modular exponentiation, which is a key step in **Shor's algorithm**.

- **Arguments:**
  - `qc`: The quantum circuit to modify.
  - `a`: The base of the exponentiation (`a^x mod N`).
  - `N`: The number to be factored.
  - `source`: The qubits representing the exponent (`x`).
  - `target`: The qubits where the modular exponentiation result is stored.

- **Implementation Details:**
  - The function currently applies only **CNOT (CX) gates** as a placeholder.
  - In a full implementation, these would be replaced with **controlled modular multiplication** gates to compute \( a^x \mod N \).
  - A barrier is added for clarity in circuit visualization.

This step is essential for encoding periodicity in the quantum state, which is later extracted using the **Quantum Fourier Transform (QFT)**.


In [4]:
def modular_exponentiation(qc, a, N, source, target):
    """Simplified modular exponentiation: a^x mod N."""
    for i in range(len(source)):
        qc.cx(source[i], target[i])
    qc.barrier()

## Inverse Quantum Fourier Transform (IQFT) Circuit

This function **`iqft_circuit`** implements the **Inverse Quantum Fourier Transform (IQFT)** on a given quantum circuit.

- **Arguments:**
  - `qc`: The quantum circuit to modify.
  - `qubits`: The list of qubits on which IQFT is applied.
  - `n`: The number of qubits.

- **Implementation Details:**
  - Applies a sequence of controlled phase gates (`cu1`) with exponentially decreasing angles.
  - Applies a Hadamard gate (`h`) on each qubit.
  - Inserts a barrier for visualization and separation.

The IQFT is crucial for extracting periodicity in Shor's algorithm, which helps in finding the factors of a given integer.


In [5]:
def iqft_circuit(qc, qubits, n):
    for i in range(n):
        for j in range(i):
            qc.cu1(-math.pi / (2 ** (i - j)), qubits[j], qubits[i])
        qc.h(qubits[i])
    qc.barrier()

### Shor's algorithm implementation

## Shor's Algorithm for Factoring a Semiprime \( N \)

This function **`shors_algorithm`** implements **Shor's algorithm** to factorize a given semiprime \( N \) using quantum computing.

### **Steps:**
1. **Choose a Random Base \( a \) (if not provided):**
   - A random base \( a \) is selected such that \( \gcd(a, N) = 1 \).
   - If the chosen base shares a common factor with \( N \), it is re-selected.

2. **Quantum Register Initialization:**
   - The number of qubits is dynamically allocated based on \( N \).
   - `source_qubits`: Holds the exponent (`x`).
   - `target_qubits`: Stores the result of modular exponentiation.
   - `classical_register`: Stores measurement results.

3. **Superposition Initialization:**
   - Hadamard gates are applied to `source_qubits` to create an equal superposition.
   - The target register is initialized to \(|1\rangle\).

4. **Modular Exponentiation:**
   - Calls `modular_exponentiation(qc, base, N, source_qubits, target_qubits)`, which encodes \( a^x \mod N \) into the quantum state.

5. **Apply Inverse Quantum Fourier Transform (IQFT):**
   - Calls `iqft_circuit(qc, source_qubits, n)` to extract periodicity.

6. **Measurement and Execution:**
   - The `source_qubits` are measured into the `classical_register`.
   - The circuit is executed on the **Quantum Rings backend** with `shots` repetitions.

7. **Interpretation of Results:**
   - Calls `interpret_quantum_counts(counts, N, base)` to extract periodicity and compute the prime factors.

### **Output:**
- The function returns the two prime factors of \( N \) by analyzing the measurement results.

This implementation is designed for execution on a **Quantum Rings** backend and provides a structured approach to quantum factorization.


In [6]:
def shors_algorithm(N, base=None):
    """Shor's algorithm to factorize any semiprime N."""
    if base is None:
        base = random.randint(2, N - 1)  # Choose a random base a
        print(base)
        while gcd(base, N) != 1:  # Ensure base is coprime with N
            base = random.randint(2, N - 1)

    # Dynamic qubit allocation
    n = math.ceil(math.log2(N))
    num_qubits = 2 * n + 3  # Source and target registers
    source_qubits = QuantumRegister(n, 'source')
    target_qubits = QuantumRegister(n + 1, 'target')
    classical_register = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(source_qubits, target_qubits, classical_register)

    # Initialize source register in superposition
    for i in range(n):  # Fix: Iterate over the range of qubits
        qc.h(source_qubits[i])
    qc.x(target_qubits[-1])  # Initialize target register to |1>
    qc.barrier()

    # Modular exponentiation: a^x mod N
    modular_exponentiation(qc, base, N, source_qubits, target_qubits)

    # Apply inverse QFT to the source register
    iqft_circuit(qc, source_qubits, n)

    print("Gate count:", qc.count_ops())


    # Measure the source register
    for i in range(n):
        qc.measure(source_qubits[i], classical_register[i])
    qc.barrier()

    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # Interpret the results to find factors
    return interpret_quantum_counts(counts, N, base)

### Method for interpreting quantum results from shor's algorithm

## Interpreting Quantum Measurement Results for Factorization

This function **`interpret_quantum_counts`** processes quantum measurement results to extract the **prime factors** of \( N \).

### **Steps:**
1. **Sort the Measurement Outcomes:**
   - The measured bitstrings are sorted in descending order based on their frequency.

2. **Extract the Period \( r \):**
   - The bitstring with the highest count is converted from binary to an integer.
   - If the measured value is `0`, it is skipped.

3. **Verify Periodicity Condition:**
   - If \( r \) is even and satisfies \( a^{r/2} \not\equiv -1 \mod N \), then:
     - Compute \( x = a^{r/2} \mod N \).
     - Compute the factors as \( \gcd(x+1, N) \) and \( \gcd(x-1, N) \).

4. **Return the Factors:**
   - If valid factors are found, they are returned as a tuple.
   - Otherwise, `(None, None)` is returned, indicating failure.

### **Purpose:**
- This function extracts periodicity from quantum results and applies post-processing to determine the prime factors.


In [7]:
def interpret_quantum_counts(counts, N, base):
    """Interpret quantum measurement counts to determine the prime factors of N."""
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    for bitstring, _ in sorted_counts:
        measured_value = int(bitstring, 2)
        if measured_value == 0:
            continue  # Skip case

        r = measured_value
        if r is None:
            continue

        # Check if r is even and a^(r/2) != -1 mod N
        if r % 2 == 0:
            x = pow(base, r // 2, N)
            if x != N - 1:
                factor1 = gcd(x + 1, N)
                factor2 = gcd(x - 1, N)
                if factor1 != 1 and factor2 != 1:
                    return (factor1, factor2)

    return (None, None)  # If no valid factors were found

### Load in the semiprime numbers

In [8]:
N = {
    8: 143,
    10: 899,
    12: 3127,
    14: 11009,
    16: 47053,
    18: 167659,
    20: 744647,
    22: 3036893,
    24: 11426971,
    26: 58949987,
    28: 208241207,
    30: 857830637,
    32: 2776108693,
    34: 11455067797,
    36: 52734393667,
    38: 171913873883,
    40: 862463409547,
    42: 2830354423669,
    44: 12942106192073,
    46: 53454475917779,
    48: 255975740711783,
    50: 696252032788709,
    52: 3622511636491483,
    54: 15631190744806271,
    56: 51326462028714137,
    58: 217320198167105543,
    60: 827414216976034907,
    62: 3594396771839811733,
    64: 13489534701147995111,
    66: 48998116978431560767,
    68: 220295379750460962499,
    70: 757619317101213697553,
    72: 4239706985407101925109,
    74: 13081178794322790282667,
    76: 48581232636534199345531,
    78: 263180236071092621088443,
    80: 839063370715343025081359,
    82: 3145102596907521247788809,
    84: 13410747867593584234359179,
    86: 74963308816794035875414187,
    88: 196328049947816898123437813,
    90: 900212494943030042797046797,
    92: 3408479268382267351010110507,
    94: 13410207519922000104514406009,
    96: 56540697284955642837798912007,
    98: 212736089539904961817389577063,
    100: 793334180624272295351382130129,
    102: 3680428259937415378335285504779,
    104: 16332602937710799037362680335351,
    106: 57831217106245162293092061499909,
    108: 248282609001868585591158742505413,
    110: 1052209925061489272435010659874991,
    112: 3787041098266201578641927407194191,
    114: 13439273072951974276382903784255637,
    116: 53125759235002964322304682027959157,
    118: 291658670780267526191653438575411491,
    120: 1153151809972770124185028131269906357,
    122: 4932188172696454339739318297923925849,
    124: 17356117513473888567381746939296160477,
    126: 70736588847140822442757134113822365169,
    128: 212537944946563703298274772990879967689,
    130: 1084744344991496578890012624928207712213,
    132: 4047187468515523114449296638529157815821,
    134: 17544816517388983161547445055372483694669,
    136: 68909661794716346033524625875715931123259,
    138: 260300599328718051119938934425302978167179,
    140: 999828801740135651717021198616667990536367,
    142: 3355483482297520282820961102046248621459173,
    144: 16631837140942852587293950352766905297528897,
    146: 85937541327755603528549497212629785145477713,
    148: 244379668284326186252863240169712092438850161,
    150: 1063901349880514705720387916171354938099705083,
    152: 4094301101616740933345553372923577469734725267,
    154: 18540363865879151504375215350972044662106182549,
    156: 74276654746024856066871733818210517618023005407,
    158: 275275419272426358674813929221957447870184635413,
    160: 1014737718417810170242113026859620408620227447581,
    162: 4478072308913255100498385793781543335816717273939,
    164: 16727777314454032963341593941190650285738385202361,
    166: 69590306463255114448417962649373740941080626000523,
    168: 315687739381804055710020885538727496795372933617227,
    170: 1267155784800316161358824961914132766229147990344997,
    172: 4010354160858905534730018630654476125957281838169269,
    174: 14153584803038660862653346883739092747954923774660251,
    176: 77432705949923513909636069140456666815340352624073251,
    178: 339852994275131877601341971143799720360724856306184203,
    180: 1179359789143995137143701718442354039063066637950737429,
    182: 4919304328219861879269200803487336608500936862015264459,
    184: 17505199067294797183746989001423060541688565870542696723,
    186: 69942997811183689049499239546127943802272331478524805397,
    188: 261274809295595261399973998023978896104786545957568559137,
    190: 1406369495880278908988444017973006804779746131711977427971,
    192: 5096174527932196609747662866401954341497716363174035275761,
    194: 17829232438088717947765479975403534458051445077992682449561,
    196: 65066066269192108839947861329734188452463615549935803519143,
    198: 349950642518780039677569542727000637357372102125566743672291,
    200: 1292650905825941096257239453478790385594125558306176183893071,
    202: 5212568709703227409964986200323079271990196603993590694680391,
    204: 24456849430211698827484825800200226142835068670603824696509853,
    206: 86484458957105177897678864034678638961222781895382346786419167,
    208: 282008119289803047168077576239866793179892385097797258038424567,
    210: 1204782513175327651249234385485579508935520775091508304137907593,
    212: 4557491279500788785092872521598135415310018444227180886738032603,
    214: 20157487867138959990956400818670113252626858623502858096773250117,
    216: 76043131885256815931216222672122360339414261530624664568946162829,
    218: 344050409069283777519808151926058652189426586409858940845468981861,
    220: 1261954136810361898882562305061560803960332696773429051013499670427,
    222: 4967098631546029459529052889119752620821846506121675016252303705879,
    224: 16820438383635236108780123377884102235161339126337922957523995233529,
    226: 89004872228161109668642757451109313028486456927748001177934114653617,
    228: 417997545002926789713327767706388156900167929618181881789066408078713,
    230: 1448119400401161666430996998583433602188054714520957809331008374203811,
    232: 6085700232501070308789059668383263517284617763227080242662586334468949,
    234: 18761475251108592243772325275525370672745869289561231704528591592371501,
    236: 75378971952862314631435911840791163346363206779933878820532663796306607,
    238: 334251059867417571197273497079020101091157708427514612078154549573744053,
    240: 1293224958516444312449612523948303217126432531171026209139782551333088281,
    242: 5936906321010438090569400987846435049778779894646896974880143196051640311,
    244: 20576188160125558108332221675210966458468072966170808188722656711461127553,
    246: 80363125512735611009514922977138951394216679592881179768811803231929707129,
    248: 371791977264476193233793691135938456442939709119625615853629510058608968387,
    250: 1042050145513045807703000231691018818256527018925081399314153802338188332967,
    252: 5279774839958419063483802629238621577728731461590896999443064285556245912531,
    254: 20335945572378210037443669733175394361807801065230296579184489046120469661507,
    256: 104343749483876530238735961449384774965065842501756473864398652143393515842787,
}

## Benchmarking Shor’s Algorithm on Different Numbers

This cell runs **Shor’s algorithm** on a set of test numbers and records the execution time and qubit usage.

### **Steps:**
1. **Initialize Storage:**
   - `timing_results`: Stores execution times for each test case.
   - `qubit_counts`: Stores the number of qubits used.

2. **Iterate Over Test Numbers:**
   - The test numbers are extracted from the `N` dictionary.
   - For each number:
     - **Compute the Required Qubits:**  
       The number of qubits needed is calculated as `2 * n + 3`, where `n = ceil(log2(N))`.
     - **Measure Execution Time:**  
       The algorithm’s runtime is measured using `time.time()` before and after execution.
     - **Attempt Factorization (Up to 10 Times):**  
       - Runs `shors_algorithm(number)`, retrying if the factors are `(None, None)`.
       - If valid factors are found, the loop terminates early.

3. **Print and Store Results:**
   - Displays the number being tested, its factors, execution time, and qubits used.
   - Appends execution time and qubit count to the respective lists.

### **Purpose:**
- This benchmarking process helps evaluate the **performance** of Shor’s algorithm on different numbers, analyzing:
  - **Time Complexity**
  - **Qubit Resource Requirements**
  - **Success Rate Across Multiple Attempts**


In [9]:
# Initialize timing results storage
timing_results = []
qubit_counts = []

test_numbers = list(N.values())

for number in test_numbers:
    print(f"\nTesting number: {number}")

    # Calculate number of qubits used
    n = math.ceil(math.log2(number))
    num_qubits = 2 * n + 3
    qubit_counts.append(num_qubits)

    # Time the execution
    start_time = time.time()

    factors = (None,None)
    attempt = 0
    while factors[0] is None and factors[1] is None:
        print(f"Attempt {attempt + 1}")
        factors = shors_algorithm(number)
        if factors[0] is not None:
            break

    end_time = time.time()
    execution_time = end_time - start_time
    timing_results.append(execution_time)

    print(f"Number: {number}")
    print(f"Factors: {factors}")
    print(f"Time taken: {execution_time:.2f} seconds")
    print(f"Qubits used: {num_qubits}")


Testing number: 143
Attempt 1
138
Gate count: {'barrier': 51, 'cu1': 28, 'cx': 8, 'h': 16, 'x': 1}
Job Queued
Job Running
Job Done.
Ending Job Monitor
Number: 143
Factors: (13, 11)
Time taken: 2.02 seconds
Qubits used: 19

Testing number: 899
Attempt 1
346
Gate count: {'barrier': 63, 'cu1': 45, 'cx': 10, 'h': 20, 'x': 1}
Job Queued
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Done.
Ending Job Monitor
Number: 899
Factors: (29, 31)
Time taken: 8.01 seconds
Qubits used: 23

Testing number: 3127
Attempt 1
2790
Gate count: {'barrier': 75, 'cu1': 66, 'cx': 12, 'h': 24, 'x': 1}
Job Queued
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Done.
Ending Job Monitor
Attempt 1
1846
Gate count: {'barrier': 75, 'cu1': 66, 'cx': 12, 'h': 24, 'x': 1}
Job Queued


KeyboardInterrupt: 